In [0]:
%pip install nbformat databricks-sdk[openai]==0.38.0 dspy --quiet
dbutils.library.restartPython()

In [0]:
%run ../prompts/ml_prompts_with_DSPy

In [0]:
# %run ./GradingModule

In [0]:
# grading_module = Module(databricks.sdk.WorkspaceClient(), module_dict_ml)
print(ml_module.get_module_dict())

In [0]:
candidate_dict = [
  ("X","/Workspace/Users/vibhor.nigam@databricks.com/interview-scripts/interview-grading/example-notebooks-ml/04-Machine-Learning-X")
]

human_graded_dict = {
  "X": [5, 30, 25, 5, 0]
}
# w = databricks.sdk.WorkspaceClient()

In [0]:
lm = dspy.LM('databricks/databricks-meta-llama-3-3-70b-instruct')
dspy.settings.configure(lm=lm)

In [0]:
context_path = candidate_dict[0][1]
human_answers_list = human_graded_dict[candidate_dict[0][0]]
table_name = "users.abhay_jalisatgi.gen_ai_eval"
section = "ML"
candidate = "X"

results = ml_module.get_error_and_answer_dict(context_path, human_answers_list, table_name, section, candidate)
results

In [0]:
results['error_dict']

In [0]:
results['answers_dict'].display()